In [59]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer, *-*"

#r "nuget: Azure.AI.OpenAI"
#r "nuget: Azure.Identity"
#r "nuget: Azure"
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Data.SqlClient"

Installed Packages Azure, 1.0.6 Azure.AI.OpenAI, 2.1.0 Azure.Identity, 1.14.0 Microsoft.Data.SqlClient, 6.0.2 Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.25177.1 Newtonsoft.Json, 13.0.3

In [ ]:
#!connect mssql --kernel-name Stackoverflow --connection-string "Server=tcp:hsavransql1.database.windows.net,1433;Initial Catalog=azuredb1;Persist Security Info=False;User ID=hsavran;Password=HasanSavran90;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;""

In [60]:
ALTER TABLE Posts ADD BodyVector VECTOR(1536)

Error: Msg 2705, Level 16, State 4, Line 1
Column names in each table must be unique. Column name 'BodyVector' in table 'Posts' is specified more than once.

In [61]:
TRUNCATE TABLE Posts;

Commands completed successfully.

In [62]:
SELECT TOP 1 PostId, Title, PostBody, ViewCount, PostType, OwnerUserId, Score
 FROM Posts WHERE PostType = 'Question'

(0 rows affected)

Info: No rows were returned for query 0 in batch 0.

In [63]:

using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;
using OpenAI.Embeddings;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using System.Net.Http;
using System.Collections.ObjectModel;
using Microsoft.Data.SqlClient;


In [64]:
var openAIClient = new AzureOpenAIClient(
    new Uri("https://savranweb.openai.azure.com/"),
    new AzureKeyCredential("3Qg3P8Fw37qaN3ZtG2JJapZZVjBFTKvOIDaIYXoL5a5fduu3wz2XJQQJ99BDACYeBjFXJ3w3AAABACOGPUlQ"));
var aiclient = openAIClient.GetEmbeddingClient("embedding");

In [65]:
ReadOnlyMemory<float> GenerateVector(string text)
{    
    OpenAIEmbedding newembedding = aiclient.GenerateEmbedding(text);
    return newembedding.ToFloats();
}

In [66]:
public class Post    
{    
    public int PostId { get; set; }
    public string PostBody { get; set; }
    public string Title { get; set; }
    public int ViewCount { get; set; }
    public int OwnerUserId { get; set; }
    public string PostType { get; set; }
    public int Score { get; set; }
    public float[] bodyvector {get;set;}
}

In [67]:
var json = await new HttpClient().GetStringAsync("https://raw.githubusercontent.com/hsavran/Presentations/refs/heads/main/stackoverflow.json");
var postList = JsonConvert.DeserializeObject<List<Post>>(json);
postList.Count.Display();
var inserttable = new System.Data.DataTable("Posts");
inserttable.Columns.Add("PostId", typeof(int));
inserttable.Columns.Add("Title", typeof(string));
inserttable.Columns.Add("PostBody", typeof(string));
inserttable.Columns.Add("ViewCount", typeof(int));
inserttable.Columns.Add("PostType", typeof(string));
inserttable.Columns.Add("OwnerUserId", typeof(int));
inserttable.Columns.Add("Score", typeof(int));
inserttable.Columns.Add("bodyvector", typeof(string));

foreach (var post in postList.Where(p => p.PostType == "Question" && p.Title.Contains("SQL")).Take(1000))
{   
    var newRow = inserttable.NewRow();
    newRow["PostId"] = post.PostId;
    newRow["PostBody"] = post.PostBody;
    newRow["Title"] = post.Title;
    newRow["ViewCount"] = post.ViewCount;
    newRow["OwnerUserId"] = post.OwnerUserId;
    newRow["PostType"] = post.PostType;
    newRow["Score"] = post.Score;
    var temp = GenerateVector(post.PostBody).ToArray();
    var json = JsonConvert.SerializeObject(temp);
    newRow["bodyvector"] = json;
    //JsonConvert.SerializeObject(string.Join(",", temp));
    inserttable.Rows.Add(newRow);
}
//inserttable.Rows.Count.Display();
//inserttable.Rows[0].Display();
using (var connection = new Microsoft.Data.SqlClient.SqlConnection("Server=tcp:hsavransql1.database.windows.net,1433;Initial Catalog=azuredb1;Persist Security Info=False;User ID=hsavran;Password=HasanSavran90;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;"))
{
    connection.Open();
    using (var transaction = connection.BeginTransaction())
    {
        using (var bulkCopy = new Microsoft.Data.SqlClient.SqlBulkCopy(connection, Microsoft.Data.SqlClient.SqlBulkCopyOptions.Default, transaction))
        {
            bulkCopy.DestinationTableName = "Posts";
            bulkCopy.BatchSize = 1000;
            bulkCopy.WriteToServer(inserttable);
        }
        transaction.Commit();
    }
}

9264

In [68]:
var vectortosearch = GenerateVector("What are the most common efficiency questions about relational databases").ToArray();
var temp = JsonConvert.SerializeObject(vectortosearch);
// run the query to get the top 10 most similar posts
var cmd = @"
DECLARE @v AS VECTOR(1536);
SET @v = (SELECT CAST(@temp AS VECTOR(1536)))
SELECT TOP(10)   
  title, postbody,
  VECTOR_DISTANCE('cosine', @v, bodyvector) AS distance
FROM 
  Posts
ORDER BY distance ";
  var texttosummary  ="";
using (var connection = new Microsoft.Data.SqlClient.SqlConnection("Server=tcp:hsavransql1.database.windows.net,1433;Initial Catalog=azuredb1;Persist Security Info=False;User ID=hsavran;Password=HasanSavran90;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;"))
{
    connection.Open();
    using (var command = new Microsoft.Data.SqlClient.SqlCommand(cmd, connection))
    {
        command.Parameters.AddWithValue("@temp", temp);
        using (var reader = command.ExecuteReader())
        {
            while (reader.Read())
            {
              texttosummary += reader["postbody"].ToString() + " ";
                Console.WriteLine($"Title: {reader["title"]}, Distance: {reader["distance"].ToString()}");               
              
            }
        }
    }
}

Title: How big can a MySQL database get before performance starts to degrade, Distance: 0.16720164433149232
Title: ADO.NET Connection Pooling & SQLServer, Distance: 0.17804568534468468
Title: Automatically measure all SQL queries, Distance: 0.19333204080520694
Title: What are the advantages of explicit Join Transitive Closure in SQL?, Distance: 0.1998784809196218
Title: LINQ-to-SQL vs stored procedures?, Distance: 0.2029639149425464
Title: Oracle XE or SQL Server Express, Distance: 0.2044463349202461
Title: What is a "reasonable" length of time to keep a SQL cursor open?, Distance: 0.20456216392069126
Title: Automate Syncing Oracle Tables With MySQL Tables, Distance: 0.20488238075016363
Title: Migrating from MySQL to PostgreSQL, Distance: 0.20550847288033303
Title: Have you ever encountered a query that SQL Server could not execute because it referenced too many tables?, Distance: 0.2057512291387743
